![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description 
|--------|-----------|-------------
| `client_id` | `integer` | Client ID 
| `age` | `integer` | Client's age in years
| `job` | `object` | Client's type of job 
| `marital` | `object` | Client's marital status 
| `education` | `object` | Client's level of education 
| `credit_default` | `bool` | Whether the client's credit is in default 
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) 

<br>

## `campaign.csv`

| column | data type | description 
|--------|-----------|-------------
| `client_id` | `integer` | Client ID 
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign 
| `contact_duration` | `integer` | Last contact duration in seconds 
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign
| `previous_outcome` | `bool` | Outcome of the previous campaign 
| `campaign_outcome` | `bool` | Outcome of the current campaign 
| `last_contact_date` | `datetime` | Last date the client was contacted

<br>

## `economics.csv`

| column | data type | description 
|--------|-----------|-------------
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator)
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator)

In [143]:
import pandas as pd
import numpy as np

# import the original csv file
bank_marketing_file = pd.read_csv("bank_marketing.csv")

# try importing only selected columns in the csv file
client = pd.read_csv("bank_marketing.csv", usecols=['client_id','age','job','marital','education','credit_default','mortgage'], index_col ='client_id')
client.info()

campaign = pd.read_csv("bank_marketing.csv", usecols=['client_id','number_contacts','contact_duration','previous_campaign_contacts','previous_outcome','campaign_outcome','month','day'], index_col = 'client_id')
campaign.info()

economics =  pd.read_csv("bank_marketing.csv", usecols=['client_id','cons_price_idx','euribor_three_months'], index_col='client_id')
economics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41188 entries, 0 to 41187
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             41188 non-null  int64 
 1   job             41188 non-null  object
 2   marital         41188 non-null  object
 3   education       41188 non-null  object
 4   credit_default  41188 non-null  object
 5   mortgage        41188 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   month                       41188 non-null  object
 1   day                         41188 non-null  int64 
 2   contact_duration            41188 non-null  int64 
 3   number_contacts             41188 non-null  int64 
 4   previous_campaign_contacts  41188 non-nul

In [144]:
# Clean the clients_df files

# Clean the job and education fields to replace "." with "_"
client['job'].replace('\.','_', inplace = True, regex = True)
client['education'].replace('\.','_',inplace = True, regex = True)

# Clean the education field to replace "Unknown" with NaN
client['education'].replace('unknown',np.NaN,inplace = True)

# Add a boolean style credit_default and mortgage fields to 1 if 'yes' and 0 otherwise and change dtype to boolean
client['credit_default_boolean'] = client['credit_default'].apply(lambda x: 1 if x == 'yes' else 0).astype('bool')
client['mortgage_boolean'] = client['mortgage'].apply(lambda x: 1 if x == 'yes' else 0).astype('bool')

#Drop the orignal credit default and mortgage columns and rename the respective boolean columns I created
client.drop(['credit_default','mortgage'], axis=1, inplace=True)
client.rename(columns={'credit_default_boolean':'credit_default','mortgage_boolean': 'mortgage'}, inplace=True)

client.head()

,age,job,marital,education,credit_default,mortgage
client_id,,,,,,
0,56,housemaid,married,basic_4y,False,False
1,57,services,married,high_school,False,False
2,37,services,married,high_school,False,True
3,40,admin_,married,basic_6y,False,False
4,56,services,married,high_school,False,False


In [145]:
import pandas as pd

# Assuming campaign_df is the DataFrame that needs to be cleaned
campaign = campaign.copy()

# Add a boolean style previous_outcome and campaign outcome fields to 1 if 'yes' and 0 otherwise and change dtype to boolean
campaign['previous_outcome'] = campaign['previous_outcome'].map({'success':1, 'failure':0, 'nonexistent':0})
campaign['campaign_outcome'] = campaign['campaign_outcome'].map({'yes':1, 'no':0})

campaign['previous_outcome'] = campaign['previous_outcome'].astype('bool')
campaign['campaign_outcome'] = campaign['campaign_outcome'].astype('bool')

# Create a datetime column from the month, day, and year column names last_contact_date
campaign['year'] = 2022
campaign['month'] = campaign['month'].str.capitalize()  # Capitalize month names
campaign['last_contact_date'] = pd.to_datetime(campaign[['year','month','day']].astype(str).agg('-'.join, axis=1))

# Drop the unneeded columns and rename the boolean columns
campaign.drop(['year','month','day'], axis=1, inplace=True)
campaign.head()

,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
client_id,,,,,,
0,261,1,0,False,False,2022-05-13
1,149,1,0,False,False,2022-05-19
2,226,1,0,False,False,2022-05-23
3,151,1,0,False,False,2022-05-27
4,307,1,0,False,False,2022-05-03


In [146]:
#Economics table requires no cleaning

#Ready to export all the files to csv
client.to_csv('client.csv', index=True)
campaign.to_csv('campaign.csv', index=True)
economics.to_csv('economics.csv', index=True)